In [13]:
import numpy as np
import pandas as pd
from ast import literal_eval

# load data
datafile_path_v3 = "./data/embeddings_10k_v3.csv"

df = pd.read_csv(datafile_path_v3)
# df_v1 = pd.read_csv(datafile_path)
# df = pd.read_csv(datafile_path_v2)
# print(df.head(10))
# df['category_encoded'] = df_v1['category_encoded']

df["embedding_cat_headline"] = df.embedding_cat_headline.apply(literal_eval).apply(np.array)  # convert string to numpy array
cat_matrix = np.vstack(df.category_encoded.values)
matrix = np.vstack(df.embedding_cat_headline.values)
cat_matrix.shape



(9750, 1)

In [14]:
df.columns

Index(['Unnamed: 0', 'category_encoded', 'embedding_cat_headline',
       'embedding_description'],
      dtype='object')

In [3]:
df["category_encoded"].unique()

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41], dtype=int64)

In [6]:
n_clusters = 42
init_centroids = []
for i in range(n_clusters):
    init_centroids.append(df[df['category_encoded'] == i].sample(150).embedding_cat_headline.mean())
init_centroids = np.array(init_centroids)
init_centroids.shape

# for i in range(n_clusters):
#     print(df[df['category_encoded'] == i].count())

(42, 1536)

In [7]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=n_clusters, init=init_centroids, random_state=7)
kmeans.fit(matrix)
labels = kmeans.labels_
df["Cluster"] = labels

# print(df.groupby("Cluster").apply(lambda x: x)['category_encoded'])
df.groupby("Cluster").apply(lambda x: x)['category_encoded'].to_csv('./data/cluster_category.csv')

# cluster_cat_df = pd.read_csv('./data/cluster_category.csv')
# cluster_cat_count = []
# for i in range(n_clusters):
#     cluster_cat_count.append(cluster_cat_df.loc[cluster_cat_df['Cluster'] == i].category_encoded.value_counts())
    # print("cluster num:", i)
    # print("total count: ", cluster_cat_df.loc[cluster_cat_df['Cluster'] == i].count())
    # print(cluster_cat_df.loc[cluster_cat_df['Cluster'] == i].category_encoded.value_counts())
# print(cluster_cat_count)
# for cluster, cat in df.groupby("Cluster").apply(lambda x: x)['category_encoded']:
    

# print(df.groupby("Cluster").apply(lambda x: x.category_encoded.value_counts().idxmax()))
cluster_category_mapping = df.groupby("Cluster").apply(lambda x: x.category_encoded.value_counts().idxmax()).to_dict()
print(cluster_category_mapping)
cluster_centers = df.groupby("Cluster").embedding_cat_headline.mean()
# cluster_centers


c:\Users\EmilyXia\anaconda3\envs\tensor_py3.8\lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
c:\Users\EmilyXia\anaconda3\envs\tensor_py3.8\lib\site-packages\sklearn\cluster\_kmeans.py:1416: RuntimeWarning: Explicit initial center position passed: performing only one init in KMeans instead of n_init=10.
  super()._check_params_vs_input(X, default_n_init=10)


{0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6, 7: 7, 8: 8, 9: 9, 10: 10, 11: 11, 12: 12, 13: 13, 14: 14, 15: 15, 16: 16, 17: 17, 18: 18, 19: 19, 20: 20, 21: 21, 22: 22, 23: 23, 24: 24, 25: 25, 26: 26, 27: 27, 28: 28, 29: 29, 30: 30, 31: 31, 32: 32, 33: 33, 34: 34, 35: 35, 36: 36, 37: 37, 38: 38, 39: 39, 40: 40, 41: 41}


In [8]:
valuess = []
for key, value in cluster_category_mapping.items():
    valuess.append(value)

print(set( valuess))
print(len(set( valuess)))


{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41}
42


In [9]:
# choose several near classes if the distances are smaller than the threshold
def choose_near_classes(embed, num_near_classes, threshold, class_centers, category_encoded):
    chosen_classes = []
    for i, center in enumerate(class_centers):
        dis = np.linalg.norm(embed - center)
        if dis < threshold and i != category_encoded:
            chosen_classes.append([i, dis])
    chosen_classes = sorted(chosen_classes, key=lambda x: x[1])
    home_dis = np.linalg.norm(embed - class_centers[category_encoded][0])
    return (chosen_classes[:num_near_classes], home_dis)

# output the class feature value: 1.0 -> dis btw doc and home class center, value -> dis btw doc and other class center/ dis btw doc and home class center
def get_class_feature_value(near_classes, home_dis, num_classes, category_encoded):
    doc_topic_feature = np.zeros(num_classes)
    doc_topic_feature[category_encoded] = 1.0
    for i in near_classes:
        doc_topic_feature[i[0]] = (home_dis / (i[1]))**2
        if doc_topic_feature[i[0]] >= 1.0:
            doc_topic_feature[i[0]] = 1.0
    return doc_topic_feature

In [10]:
near_classes = choose_near_classes(matrix[10], 10, 0.6, cluster_centers.values, cat_matrix[10])
print(near_classes[0])
print(near_classes[1])
doc_topic_feature = get_class_feature_value(near_classes[0], near_classes[1], 42, cat_matrix[10])
print(doc_topic_feature)

all_doc_topic_feature = []
# print(len(all_doc_topic_feature[0]))
for i, x in enumerate(matrix):
    near_classes = choose_near_classes(x, 4, 0.6, cluster_centers.values, cat_matrix[i])
    doc_topic_feature = get_class_feature_value(near_classes[0], near_classes[1], n_clusters, cat_matrix[i])
    all_doc_topic_feature.append(doc_topic_feature)
# save to csv file && add header
df_topic = pd.DataFrame(all_doc_topic_feature)
df_topic.to_csv('./data/doc_vector_feature.csv', index=False)

# # test outcome
df_topic = pd.read_csv('./data/doc_vector_feature.csv', index_col=False)
features = np.array(df_topic.values)
print(features.shape)


[[7, 0.42284585724456525], [1, 0.43081582513866223], [36, 0.46456037163918334], [8, 0.5043451942637044], [29, 0.5087263184723162], [26, 0.5150867067190147], [39, 0.5171821479971169], [17, 0.5258631482908701], [10, 0.5277651426621888], [23, 0.5284401116819762]]
0.3871317669539723
[1.         0.80748474 0.         0.         0.         0.
 0.         0.8382112  0.58919879 0.         0.53806682 0.
 0.         0.         0.         0.         0.         0.54196613
 0.         0.         0.         0.         0.         0.53669317
 0.         0.         0.56488096 0.         0.         0.57909419
 0.         0.         0.         0.         0.         0.
 0.69443771 0.         0.         0.56031283 0.         0.        ]
(9750, 42)
